In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.transforms as T
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import transform
from skimage import filters
from skimage import color
from PIL import Image
import numpy as np
import imageio
import skimage
import glob

In [ ]:
dtype = torch.float32
device = torch.device('cpu')

In [ ]:
def load_pics():
    
    X = []
    Y = []
    Z = []
    f = open('micheI_sgs4.txt')
       
    counter = 0
    for idx, l in enumerate(f):e
        l = l.strip().split(' ')
        name, x, y, r = l[0], int(l[1]), int(l[2]), int(l[3])
        pic = imageio.imread('data/BipLab/SamsungGalaxyS4/' + name + '.jpg')
        h, w, c = pic.shape
        scale_factor = 160.0/h
        pic_small = skimage.transform.resize(pic, (160,90,3))
        new_r = int(r * scale_factor)
        new_x = int(x * scale_factor)
        new_y = int(y * scale_factor)
        X.append(pic_small)
        Y.append(new_r)
        Z.append((new_x, new_y))
        
        if counter % 50 == 0:
            print(counter)
            fig, ax = plt.subplots(1)
            plt.imshow(pic_small)
            rect_true = patches.Rectangle((new_x-new_r,new_y-new_r),2*new_r,2*new_r, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect_true)
            plt.show()
            
        counter += 1
            
    return (X, Y, Z)

In [ ]:
def gen_sets(X, Y, Z):
    
    # Convert to np-arrays
    X_np = np.array(X)
    Y_np = np.array(Y)
    print("Gen Sets: Original Shapes")
    print(X_np.shape)
    print(Y_np.shape)

    # Reshape to Correect Tensor Format
    Xt = X_np.reshape((X_np.shape[0], 1, X_np.shape[3], X_np.shape[1], X_np.shape[2]))
    Yt = Y_np.reshape(Y_np.shape[0], 1, 1)
    print("Gen Sets: Reshaped Shapes")
    print(Xt.shape)
    print(Yt.shape)
    
    # Define Split Boundaries
    N = Xt.shape[0]
    train = int(6*N/10)
    val = int(7*N/10)
    
    # Train-Val-Test Partitioning
    X_train = Xt[:train]
    y_train = Yt[:train]
    
    X_val = Xt[train:val]
    y_val = Yt[train:val]
    
    X_test = Xt[val:]
    y_test = Yt[val:]
    Z_test = Z[val:]
    
    # Convert to Torch Tensors
    X_train = torch.tensor(X_train).type(torch.FloatTensor)
    y_train = torch.tensor(y_train)
    X_val = torch.tensor(X_val).type(torch.FloatTensor)
    y_val = torch.tensor(y_val)
    X_test = torch.tensor(X_test).type(torch.FloatTensor)
    y_test = torch.tensor(y_test)

    # Report Shapes
    print(X_train.shape)
    print(y_train.shape)
    print(X_val.shape)
    print(y_val.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    return (X_train, y_train, X_val, y_val, X_test, y_test, Z_test)

In [ ]:
def check_accuracy(mode, model, verbose=False, margin=0):
            
    if mode == 'val':
        X_sel = X_val
        y_sel = y_val
    elif mode == 'train':
        X_sel = X_train
        y_sel = y_train
    else:
        X_sel = X_test
        y_sel = y_test
    
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for t in range(X_sel.shape[0]):
            
            x = X_sel[t]
            y = y_sel[t]
            
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            _, preds = scores.max(1)
            if abs(preds - y) <= margin:
                num_correct += 1
            num_samples += preds.size(0)
            
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f) for mode %s' % (num_correct, num_samples, 100 * acc, mode))
    return 100*acc

In [ ]:
def train(model, optimizer, epochs=20):
    
    model = model.to(device=device)
    margins = 3
    train_accs = [[] for _ in range(margins)]
    val_accs = [[] for _ in range(margins)]
    for e in range(epochs):
        for t in range(X_train.shape[0]):
            model.train() 
            
            x = X_train[t]
            y = y_train[t]
            
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y[0])
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            if t % 200 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print() 
        
        print('epoch %d' % (e))
        for m in range(margins):
            print('With margin {}:'.format(m))
            train_accs[m].append(check_accuracy('train', model, margin=m))
            val_accs[m].append(check_accuracy('val', model, margin=m))
        print()
    fig, ax = plt.subplots(1, margins, sharey=True)
    for m in range(margins):
        ax[m].plot(train_accs[m])
        ax[m].plot(val_accs[m])
    plt.show()

In [ ]:
def flatten(x):
    return x.view(x.shape[0] , -1)

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [ ]:
def train_model():
    
    channel_1 = 32
    channel_2 = 32
    channel_3 = 32
    channel_4 = 16
    channel_5 = 16

    model1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=10, padding=0, stride=10),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

        Flatten(),
        nn.Linear(2048, 625),
        nn.ReLU(),
        nn.Linear(625, 45)
    )
    
    model2 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=5, padding=0, stride=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(64, 16, 3, padding=1),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

        Flatten(),
        nn.Linear(13104, 1000),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1000, 45)
    )
    
    model3 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=5, padding=0, stride=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 32, 3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 16, 3, padding=0),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

        Flatten(),
        nn.Linear(2592, 676),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(676, 45)
    )
    
    model4 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=5, padding=0, stride=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 32, 3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 16, 3, padding=0),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

        Flatten(),
        nn.Linear(2592, 676),
        nn.ReLU(),
        nn.Dropout(0.5),
        
        nn.Linear(676, 100),
        nn.ReLU(),
        nn.Dropout(0.5),
        
        nn.Linear(100, 45)
    )
    
    model5 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=5, padding=0, stride=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 32, 5, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        
        nn.Conv2d(32, 16, 3, padding=0),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

        Flatten(),
        nn.Linear(2592, 676),
        nn.ReLU(),
        nn.Dropout(0.5),
        
        nn.Linear(676, 100),
        nn.ReLU(),
        nn.Dropout(0.5),
        
        nn.Linear(100, 45)
    )

    #optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, nesterov=True)
    optimizer1 = optim.Adam(model1.parameters(), lr=1e-4, betas=(0.9, 0.999))
    optimizer2 = optim.Adam(model2.parameters(), lr=1e-4, betas=(0.9, 0.999))
    optimizer3 = optim.Adam(model3.parameters(), lr=1e-4, betas=(0.9, 0.999))
    optimizer4 = optim.Adam(model4.parameters(), lr=1e-4, betas=(0.9, 0.999))
    optimizer5 = optim.Adam(model5.parameters(), lr=1e-4, betas=(0.9, 0.999))
    train(model1, optimizer1, epochs=20)
    train(model2, optimizer2, epochs=20)
    train(model3, optimizer3, epochs=20)
    train(model4, optimizer4, epochs=20)
    train(model5, optimizer5, epochs=20)
    return [model1, model2, model3, model4, model5]

In [ ]:
def evaluation(Z_test, model):
    
    dist = []
    for image_idx in range(X_test.shape[0]):

        (x, y) = Z_test[image_idx]
        r_true = y_test[image_idx].item()
        r_pred = np.argmax(np.array(model(X_test[image_idx]).data))
        dist.append(abs(r_true - r_pred))
       
        if(image_idx % 100 == 0):
            fig, ax = plt.subplots(1)
            plt.imshow(X_test[image_idx,0,:,:,:].reshape(160,90,3))
            rect_true = patches.Rectangle((x-r_true,y-r_true),2*r_true,2*r_true, linewidth=3, edgecolor='r', facecolor='none')
            rect_pred = patches.Rectangle((x-r_pred,y-r_pred),2*r_pred,2*r_pred, linewidth=1, edgecolor='b', facecolor='none')
            ax.add_patch(rect_true)
            ax.add_patch(rect_pred)
            plt.show()

    return dist

In [ ]:
# Load Initial Datasets
X, Y, Z = load_pics()

In [ ]:
# Train the Model
X_train, y_train, X_val, y_val, X_test, y_test, Z_test = gen_sets(X, Y, Z)
models = train_model()
for model in models:
    check_accuracy('test', model)

In [ ]:
for model in models:
    check_accuracy('test', model, margin=0)
    check_accuracy('test', model, margin=1)
    check_accuracy('test', model, margin=2)
    print()

In [ ]:
# Update the Datasets with New Negative Examples
for model in models:
    dist = evaluation(Z_test, model)
    plt.hist(np.array(dist))
    plt.title('Training Set Model Analysis')
    plt.xlabel('Euclidean Distances to Ground-Truth Radius')
    plt.ylabel('Frequency')
    plt.show()